In [1]:
import numpy as np
import math as m
import os
import random
import pandas as pd
import time
import copy
import warnings
import shutil
import re
import sys
import threading
import datetime
import trilogy
import astroalign as aa
from collections import Counter
from skimage import transform as tf

import matplotlib as mpl
import matplotlib.colors as mcol
import matplotlib.patches as patches
from matplotlib import pyplot as plt, cm
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,NavigationToolbar2Tk)

import astropy
from astropy.io import fits
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy.visualization import astropy_mpl_style
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve
from astropy.stats import sigma_clipped_stats
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.modeling import models, fitting
from astropy.utils.exceptions import AstropyUserWarning
from astropy.wcs import WCS
from astropy.table import Table
from astropy.io import ascii
from shapely.geometry import Point, Polygon

from tkinter import *
from tkinter import ttk
from tkinter.ttk import Style
import tkinter.font as font
from idlelib.tooltip import Hovertip
from tkinter import filedialog

import pcigale
from pcigale.session.configuration import Configuration

from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture
from configparser import ConfigParser



# root window
root = Tk()
root.state('zoomed')
root.title("QPCMD")
root['background']='#e7e7e7'
root.lift()
root.grid_propagate(0)
mpl.use("TkAgg")

# global params:

hsc_filters = ['G','R','I','Z','Y']
sdss_filters = ['G','R','I','Z','U']
kids_filters = ['G','R','I','U']

sdss_errors = [0.0148, 0.027, 0.0458, 0.2145, 0.0502]
hsc_filternames = ['#id','redshift','subaru.hsc.g','subaru.hsc.g_err','subaru.hsc.r','subaru.hsc.r_err','subaru.hsc.i','subaru.hsc.i_err','subaru.hsc.z','subaru.hsc.z_err','subaru.hsc.y','subaru.hsc.y_err']
sdss_filternames = ['#id', 'redshift','sdss.gp','sdss.gp_err','sdss.rp','sdss.rp_err','sdss.ip','sdss.ip_err','sdss.zp','sdss.zp_err','sdss.up','sdss.up_err',]
kids_filternames = ['#id', 'redshift','sdss.gp','sdss.gp_err','sdss.rp','sdss.rp_err','sdss.ip','sdss.ip_err','sdss.up','sdss.up_err',]
ismodel = False
flag = False
msklist = [[],[],[],[]]



# plot
def plot():
    global msklist,filters,fig,fig2,fig6, fig_canvas, cad_r, ali_g_l, center, inpdataf, rd, grd, axes, dotl, nlist, df, shade,inpdata, inpdatac, ismodel,cutdata
    ismodel = False
    flag = False
    msklist = [[],[],[],[]]
    
    logl.config(text="getting input data..")
    root.update()
    
    # get input data
    inpdata = []
    inpdataerr = []
    
    # which filters should be used?
    if rbvar.get() == 1:
        filters = kids_filters
    if rbvar.get() == 2:
        filters = hsc_filters
    if rbvar.get() == 3:
        filters = sdss_filters
    
        
    for filt in filters:
        # open .fits file for filter
        inpdata.append(fits.open(inpdataf+""+filt+".fits"))
        # get header for filter
        head = fits.open(inpdataf+filt+".fits")[0].header
        #print(repr(head))
        
        
        # KiDS
        if rbvar.get() == 1:
            # get data from hdul
            inpdata[-1] = inpdata[-1][0].data
            # get the smallest flux greater than zero:
            #fmin = min(inpdata[-1][inpdata[-1]>0])
            # set all pixels with negative fluxes to fmin
            #inpdata[-1][inpdata[-1]<0] = fmin
            # get fluxes in mJy
            
            inpdata[-1] = 0.001*inpdata[-1]*10**(23.9/2.5)
            # get error in mJy
            inpdata.append(0.001*10**(23.9/2.5 + np.log10(10**((head['ABMAGLIM']/(-2.5))))))
            
        # hsc
        if rbvar.get() == 2:
            # get data from hdul
            inpdata[-1] = inpdata[-1][1].data
            # get fluxes in mJy
            inpdata[-1] = 10**(23.9/2.5)*inpdata[-1]/(fluxmag0hsc*1000)
            # get error in mJy (background variation)
            inpdata.append(10**(23.9/2.5)*np.sqrt(head['BGVAR'])/fluxmag0hsc/1000)
        
        # SDSS
        if rbvar.get() == 3:
            # get data from hdul
            inpdata[-1] = inpdata[-1][0].data
            # get the smallest flux greater than zero:
            fmin = min(inpdata[-1][inpdata[-1]>0])
            # set all pixels with negative fluxes to fmin
            inpdata[-1][inpdata[-1]<0] = fmin
            # get fluxes in mJy
            inpdata[-1] = 10**(1.4/2.5 + np.log10(inpdata[-1])/2.5)/1000
            # get error in mJy
            inpdata.append(10**(1.4/2.5 + np.log10(np.sqrt(sdss_errors[filters.index(filt)]))/2.5)/1000)
        
            
        # hsc:
        # I AB to mJy: mJy = 10^((23.9 - AB)/2.5)/1000
        # AB = 23.9 - 2.5log(1Jy) 
        # II data to AB = 2.5*log(fluxMag0/flux) <=> fluxmag0/10^(AB/2.5) = flux
        # mJy = 10^(23.9/2.5)*rawdata/(fluxmag0*1000)
        
        # KiDS
        # AB = -2.5*np.log10(rawdata/pixscalekids)
        # mJy = 10^((23.9/2.5 + np.log10(rawdata/pixscalekids)))/1000
        # log(1000*mJy) = 23.9/2.5 + log(rawdata/pixscalekids)
        # mJy = 10**(23.9/2.5 + log(rawdata/pixscalekids))/1000
        
        # SDSS
        # AB = 22.5 - np.log10(rawdata/pixscalesdss)
        # np.log10(1000*mJy) = 1.4/2.5 + np.log10(rawdata)/2.5
        # 1000*mJy*10^(-1.4/2.5) =
        # mJy = 10^(1.4/2.5 + np.log10(rawdata)/2.5)/1000
        
        # wolfram alpha:
        # mJy = 10^(1.4/2.5-3)*2.5th root of
    
    shade = [[],[],[]]
    
    
       
    
        
    if alnchk.get():
        logl.config(text="aligning images..")
        root.update()
        # align other filters to first filter
        for i in range(1,len(inpdata)):
            if inpdata[i].ndim == 2:
                inpdata[i] = np.array(aa.register(inpdata[i],inpdata[0])[0])
        inpdata[0] = np.array(inpdata[0])
        logl.config(text="aligning done.")
        root.update()
        
    
    
    # cutout
    cutdata = inpdata
    for i in range(0,len(inpdata),2):
        cutdata[i] = inpdata[i][center[1]-int(size/2 + 0.5):center[1]+int(size/2 - 0.5), center[0]-int(size/2 + 0.5):center[0]+int(size/2 - 0.5)]
    
    
   
    # rebin        
    if rebinchk.get():
        for f in range(0,len(cutdata),2):
            if len(cutdata[f])%rebins == 0:
                rebinned = [[0 for y in range(int(size/rebins))] for x in range(int(size/rebins))]
                for x in range(len(cutdata[f])):
                    for y in range(len(cutdata[f][0])):
                        rebinned[x//rebins][y//rebins] += cutdata[f][x][y]
                        
                for x in range(len(rebinned)):
                    for y in range(len(rebinned[0])):
                        rebinned[x][y] = rebinned[x][y]/rebins**2
                cutdata[f] = np.array(rebinned)
                
        
                
            else:
                print("daneben um "+str(len(cutdata[f])%rebins))
                print(len(cutdata[f]),rebins)
    
    # convolve
    if convchk.get():
            
        
        kernel = Gaussian2DKernel(x_stddev=1,x_size=convs,y_size=convs)
        logl.config(text="convolving images..")
        root.update()
        for i in range(0,len(cutdata),2):
            cutdata[i] = convolve(inpdata[i], kernel,preserve_nan=True)
        logl.config(text="convolving done.")
        root.update()
        
    
    
    
    # apply mask from config file
    
    # split the string from the config file to a list
    peakl = [x for x in npeak.split()]
    
    # split each element of the list into 1. x coordinate, 2. y coordinate, 3. mask size
    for i in range(0,len(peakl)):
        peakl[i] = [int(x) for x in peakl[i].split(",")]
    
    if len(peakl) > 0 and rebinchk.get():
        for i in range(0,len(peakl)):
            for j in range(0,len(cutdata),2):
                cutdata[j] = mask(cutdata[j],int(peakl[i][0]/rebins), int(peakl[i][1]/rebins),int((peakl[i][2]/rebins)**2))
    else:
        for i in range(0,len(peakl)):
            for j in range(0,len(cutdata),2):
                cutdata[j] = mask(cutdata[j],int(peakl[i][0]), int(peakl[i][1]),int(peakl[i][2]))
   
    
    
    
    
    #get magnitudes
    # KiDS
    if rbvar.get() == 1:
        for i in range(0,len(cutdata),2):
            # get the smallest flux greater than zero:
            fmin = min(cutdata[i][cutdata[i]>0])
            # set all pixels with negative fluxes to fmin
            cutdata[i][cutdata[i]<0] = fmin
            # calculate the magnitudes
            cutdata[i] = 23.9 - 2.5*np.log10(1000*cutdata[i]/pixscalekids)
        
    # hsc
    if rbvar.get() == 2:
        for i in range(0,len(cutdata),2):
            # get the smallest flux greater than zero:
            fmin = min(cutdata[i][cutdata[i]>0])
            # set all pixels with negative fluxes to fmin
            cutdata[i][cutdata[i]<0] = fmin
            # calculate the magnitudes
            cutdata[i] = 23.9 - 2.5*np.log10(1000*cutdata[i]/pixscalehsc)
            
    # SDSS
    if rbvar.get() == 3:
        for i in range(0,len(cutdata),2):
            # get the smallest flux greater than zero:
            fmin = min(cutdata[i][cutdata[i]>0])
            # set all pixels with negative fluxes to fmin
            cutdata[i][cutdata[i]<0] = fmin
            # calculate the magnitudes
            cutdata[i] = 23.9 - 2.5*np.log10(1000*cutdata[i]/pixscalesdss)
        
    
    
    

    

    # µ_g - µ_r
    grd = cutdata[0]-cutdata[2]
    darr = [[],[],[]]
    cc = 0
    dfd = {"x": [], "y": [], "g-r":[], "r": [], "colorcode": [], "alpha":[], "density":[]}
    df = pd.DataFrame(dfd)
    
    # generate data frame
    logl.config(text="generating data frame..")
    root.update()
    maxd = np.sqrt(2*(len(cutdata[2])/2)**2)
    start = time.process_time()
    for x in range(0,len(cutdata[2])):
        # countdown approximation
        t = time.process_time() - start
        d = x
        if x > 5:
            sp = t/d
            logl.config(text="generating data frame..\n"+str(x)+"/"+str(len(cutdata[2]))+" - "+str(round(100*d/len(cutdata[2]), 1))+"% - "+sectotime(sp*(len(cutdata[2])-d)))
            root.update()
        for y in range(0,len(cutdata[0])):
            if mmin < cutdata[2][x][y] < mmax and cmin < grd[x][y] < cmax :
                
                if np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) <= 0.2*maxd:
                    cc = 0
                elif np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) > 0.2*maxd and np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) <= 0.4*maxd:
                    cc = 0.25
                elif np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) > 0.4*maxd and np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) <= 0.6*maxd:
                    cc = 0.5
                elif np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) > 0.6*maxd and np.sqrt((x-len(cutdata[2])/2)**2+(y-len(cutdata[2])/2)**2) <= 0.8*maxd:
                    cc = 0.75
                else:
                    cc = 1
                new_row = {'x':x, 'y':y, 'g-r':grd[x][y], 'r':cutdata[2][x][y], 'colorcode':cc, 'alpha':1, 'density': 0}
                df = df.append(new_row, ignore_index=True)
    darrl = len(df.index)
    #print(df)
    fig = Figure(figsize=(6,5), dpi = 100)
    fig_canvas = FigureCanvasTkAgg(fig)
    
    axes = fig.add_subplot()
    xscale = 1/((axes.get_window_extent().width*72./fig.dpi)/np.sqrt(dsize)/abs(cmax-cmin))
    yscale = 1/((axes.get_window_extent().height*72./fig.dpi)/np.sqrt(dsize)/abs(mmax-mmin))
    
    
    # for density map (obsolete)
    if 1==2:
        k = 200
        ks = 7
        dmap = [[0 for x in range(k*5)] for y in range(k*6)]
        xscale = (cmax-cmin)/(k*6) #pixelscale of density map
        yscale = (mmax-mmin)/(k*5)
        
        
        i = 0
        j = 0
        start = 0
        h = 0
        if isinstance(thinning, float):
            logl.config(text="generating density map..")
            root.update()
            start = time.process_time()
            for i in range(len(df.index)):
                #x = (df["g-r"][i]-cmin)//xscale
                for u in range(-ks,ks):
                    for v in range(-ks,ks):
                        xi = int((df["g-r"][i]-cmin)//xscale + u)
                        yi = int((df["r"][i]-mmin)//yscale + v)
                        #print(xi,yi)
                        if 0 <= xi < len(dmap) and 0 <= yi < len(dmap[0]):
                            dmap[xi][yi] += 1
                    
                        
                    
                    
                if i > 0:
                    t = time.process_time() - start
                    d = i
                    sp = t/d
                    logl.config(text="generating density map..\n"+str(i)+"/"+str(len(df.index))+" - "+sectotime(sp*(len(df.index)-d)))
                    root.update()
                    
            mx = max(map(max, dmap))
            for i in range(len(dmap)):
                for j in range(len(dmap[0])):
                    if dmap[i][j] == 0:
                        dmap[i][j] = -1
                    else:
                        dmap[i][j] = dmap[i][j]/mx
            start = time.process_time()
            for i in range(len(df.index)):
                df['density'][i] = dmap[int((df['g-r'][i]-cmin)/(cmax-cmin)*k*6)][int((df['r'][i]-mmin)/(mmax-mmin)*k*5)]
                if i > 0:
                    t = time.process_time() - start
                    d = i
                    sp = t/d
                    logl.config(text="appending density map to data frame..\n"+str(i)+"/"+str(len(df.index))+" - "+sectotime(sp*(len(df.index)-d)))
                    root.update()
            
        
        

   

    logl.config(text="plotting pCMD..")
    root.update()
    
    
    axes.scatter(df["g-r"],df["r"],c=df["colorcode"], marker='s', s=dsize, vmin=0, vmax=1, cmap='viridis_r')
    axes.set_ylabel('$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_xlabel('$µ_g$-$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_ylim(mmax,mmin)
    axes.set_xlim(cmin,cmax)
    axes.set_title(title)
    fig_canvas.get_tk_widget().grid(row=4,rowspan=5,column=0,columnspan=12)
    
    dotl = Label(root, text="", font=("Arial",1), bg='red',width=2,height=1)
    
    
    
    fig6 = Figure(figsize=(6,5), dpi = 100)
        
    axes6 = fig6.add_subplot()
    axes6.scatter(df["g-r"],df["r"],c=df["density"], marker='s', s=dsize, vmin=0, vmax=1, cmap='viridis_r')
    
    axes6.set_ylabel('$µ_r$ [mag arcsec$^{-2}$]')
    axes6.set_xlabel('$µ_g$-$µ_r$ [mag arcsec$^{-2}$]')
    axes6.set_ylim(mmax,mmin)
    axes6.set_xlim(cmin,cmax)
    axes6.set_title(title)
    
    
    
    # show image
    logl.config(text="plotting image..")
    root.update()
    fig2 = Figure(figsize=(6,5), dpi = 100)
    fig2_canvas = FigureCanvasTkAgg(fig2)
    fig2.canvas.mpl_connect("motion_notify_event",hover)
    fig2.canvas.mpl_connect("button_press_event",click)
    fig2.canvas.mpl_connect("button_release_event",release)
    c1 = plt.Circle((len(cutdata[2])/2,len(cutdata[2])/2), maxd*0.2,linewidth=1, edgecolor='yellow',facecolor="none")
    c2 = plt.Circle((len(cutdata[2])/2,len(cutdata[2])/2), maxd*0.4,linewidth=1, edgecolor='green',facecolor="none")
    c3 = plt.Circle((len(cutdata[2])/2,len(cutdata[2])/2), maxd*0.6,linewidth=1, edgecolor='mediumturquoise',facecolor="none")
    c4 = plt.Circle((len(cutdata[2])/2,len(cutdata[2])/2), maxd*0.8,linewidth=1, edgecolor='blue',facecolor="none")
    
    axes2 = fig2.add_subplot()
    axes2.add_patch(c1)
    axes2.add_patch(c2)
    axes2.add_patch(c3)
    axes2.add_patch(c4)
    axes2.set_ylim(0,len(cutdata[2]))
    axes2.set_xlim(0,len(cutdata[2]))
    axes2.set_title(title)
    
    # color map
    #image1 = axes2.imshow(cutdata[0]-cutdata[4],cmap='gray',extent=(0,len(cutdata[2]),len(cutdata[2]),0))
    #colorbar = fig2.colorbar(image1)
    
    # grayscale
    axes2.imshow(cutdata[2],cmap='gray',extent=(0,len(cutdata[2]),len(cutdata[2]),0))
    
    
    fig2_canvas.get_tk_widget().grid(row=4,rowspan=5,column=12,columnspan=12)
    logl.config(text="plotting done.\n"+str(len(darr[0]))+" of "+str(darrl)+" datapoints.")

def save_plot():
    global fig,fig2,nlist,axes,axes2,foldername
    
    if ismodel:
        logl.config(text="saving model pcmd..")
        root.update()
        try:
            fig.savefig(confdir+"/"+foldername+"/"+title+'_pCMD_model.pdf')
        except:
            # Get the current date and time
            current_time = datetime.datetime.now()

            # Format the current time as a string
            foldername = current_time.strftime("%Y-%m-%d_%H-%M-%S")

            # Create a folder with the formatted name
            os.mkdir(foldername)
            
            # save fig in folder
            fig.savefig(confdir+"/"+foldername+"/"+title+'_pCMD_model.pdf')
        logl.config(text="saving model image..")
        root.update()
        if len(paramvar.get()) > 1:
            fig2.savefig(confdir+"/"+foldername+"/"+title+'_'+str(paramvar.get())+'.pdf')
        else:
            fig2.savefig(confdir+"/"+foldername+"/"+title+'_image_model.pdf')
        logl.config(text="saving color image..")
        root.update()
        color_image()
    else:
        logl.config(text="saving pcmd..")
        root.update()
        try:
            fig.savefig(confdir+"/"+foldername+"/"+title+'_pCMD.pdf')
        except:
            # Get the current date and time
            current_time = datetime.datetime.now()

            # Format the current time as a string
            foldername = current_time.strftime("%Y-%m-%d_%H-%M-%S")

            # Create a folder with the formatted name
            os.mkdir(foldername)
            
            # save fig in folder
            fig.savefig(confdir+"/"+foldername+"/"+title+'_pCMD.pdf')
        logl.config(text="saving image..")
        root.update()
        if len(paramvar.get()) > 1:
            fig2.savefig(confdir+"/"+foldername+"/"+title+'_'+str(paramvar.get())+'.pdf')
        else:
            fig2.savefig(confdir+"/"+foldername+"/"+title+'_image.pdf')
        logl.config(text="saving color image..")
        root.update()
        color_image()
    
        
    logl.config(text="saving done.")
    
def replot():
    global inp_r, inp_g, inp_rf, data_r, dotl, df, b, fig, fig2, axes, axes2, msklist, cutdata
    if b == 0:
        b = 1
        for i in range(len(df.index)):
            df["alpha"][i] = 0.1
        #df = df.drop(df[df.y < 74].index)
        #df = df.drop(df[df.y > 76].index)
        #df = df.drop(df[df.x > 75].index)
    #print(msklist)
    
    #es muss alles in shapely.polygon übersetzt werden, damit polygon.contains funktioniert (nervig)
    polygons = [0,0,0,0]
    for i in range(len(msklist)):
        for vertex in msklist[i]:
            df.loc[(df['x'] == vertex[1]) & (df['y'] == vertex[0]), ['colorcode']] = (i+1)/4-0.25
            
            vertex = (vertex[0],vertex[1])
        if i == 0:
            polygons[i] = [Polygon(msklist[i]),'yellow']
            #polygons[i] = {'vertices': msklist[i], 'color': 'yellow', 'alpha': 0.5}
        if i == 1:
            polygons[i] = [Polygon(msklist[i]), 'green']
            #polygons[i] = {'vertices': msklist[i], 'color': 'green', 'alpha': 0.5}
        if i == 2:
            polygons[i] = [Polygon(msklist[i]), 'mediumturquoise']
            #polygons[i] = {'vertices': msklist[i], 'color': 'mediumturquoise', 'alpha': 0.5}
            
        if i == 3:
            polygons[i] = [Polygon(msklist[i]), 'blue']
            #polygons[i] = {'vertices': msklist[i], 'color': 'blue', 'alpha': 0.5}
        start = time.process_time()
        for j in range(len(df.index)):
            t = time.process_time() - start
            d = j
            if j > 5:
                sp = t/d
                logl.config(text="coloring datapoints..\n"+str(d)+"/"+str(len(df.index))+" - "+str(round(100*d/len(df.index), 1))+"% - "+sectotime(sp*(len(df.index)-d)))
                root.update()
            point = Point(df['x'][j], df['y'][j])
            if polygons[i][0].contains(point):
                #print(point)
                df.loc[(df['x'] == df['x'][j]) & (df['y'] == df['y'][j]), ['alpha']] = 1
        #ich gebe auf, keine lust mehr
        
        fig = Figure(figsize=(6,5), dpi = 100)
        fig_canvas = FigureCanvasTkAgg(fig)
        
    
    
    axes = fig.add_subplot()
    axes.scatter(df["g-r"],df["r"],c=df["colorcode"], alpha=df["alpha"], marker='s', s=dsize*10, vmin=0, vmax=1, cmap='viridis_r')
    axes.set_ylabel('$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_xlabel('$µ_g$-$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_ylim(mmax,mmin)
    axes.set_xlim(cmin,cmax)
    axes.set_title(title)
    fig_canvas.get_tk_widget().grid(row=4,rowspan=5,column=0,columnspan=12)
    dotl = Label(root, text="", font=("Arial",1), bg='red',width=2,height=1)
    
    fig2 = Figure(figsize=(6,5), dpi = 100)
    fig2_canvas = FigureCanvasTkAgg(fig2)
    fig2.canvas.mpl_connect("motion_notify_event",hover)
    fig2.canvas.mpl_connect("button_press_event",click)
    fig2.canvas.mpl_connect("button_release_event",release)
    
    axes2 = fig2.add_subplot()
    axes2.set_ylim(0,len(cutdata[2]))
    axes2.set_xlim(0,len(cutdata[2]))
    axes2.set_title(title)
    shades = (axes2.get_window_extent().width*72./fig.dpi)**2/size**2-2
    
    axes2.imshow(cutdata[2],cmap='gray',extent=(0,len(cutdata[2]),len(cutdata[2]),0))
    for i in range(len(polygons)):
        if len(msklist[i])>0:
            axes2.add_patch(patches.Polygon(msklist[i], closed=True, edgecolor=polygons[i][1], facecolor=polygons[i][1], alpha=0.5))
    #hier weiter
    
    
    
    
    fig2_canvas.get_tk_widget().grid(row=4,rowspan=5,column=12,columnspan=12)
    
    
    
def plotmodel():
    thread = threading.Thread(target=plotmodel_thread)
    thread.start()
def plotmodel_thread():
    global msklist,flag, dotl, ismodel, isparamcc, modelr, modelg, modelgrd,fig, fig2, axes, axes2, df2, paramcc, filters, cig_conf, restab, my_cmap
    ismodel = True
    flag = False
    msklist = []
    # which filters should be used?
    if rbvar.get() == 1:
        filters = kids_filters
    if rbvar.get() == 2:
        filters = hsc_filters
    if rbvar.get() == 3:
        filters = sdss_filters
    
    try:
        # try loading the cigale results table
        logl.config(text="loading results..")
        #root.update()
        restabfile = fits.open(confdir+"/out/results.fits",memmap=False)
        restab = restabfile[1].data
    except:
        # if it doesnt exist, try running cigale with values file
        logl.config(text="results.fits does not exist,\nrunning cigale..")
        #root.update()
        if os.path.exists(confdir+"/values.dat"):
            # Read the configuration file
            with open("pcigale.ini", "r") as configfile:
                config_data = configfile.readlines()

            
            updated_config_data = []
            for line in config_data:
                if "ne =" in line:
                    #line = "ne = 10, 100, 1000"
                    print("line found")

                updated_config_data.append(line)

            # Write the updated configuration back to the file
            with open("pcigale.ini", "w") as configfile:
                configfile.writelines(updated_config_data)
            
            cig_conf = Configuration()
            print(cig_conf)
            
            
                
            pcigale.run(cig_conf)
        else:
            logl.config(text="cigale-data does not exist,\n creating cigale-data..")
            #root.update()
            create_cigale_values()
            logl.config(text=" running cigale..")
            #root.update()
            pcigale.run(cig_conf)
        
        restabfile = fits.open(confdir+"/out/results.fits",memmap=False)
        restab = restabfile[1].data
        
    
    # create empty model images
    modelg = np.zeros((int(size/rebins),int(size/rebins)))
    modelr = np.zeros((int(size/rebins),int(size/rebins)))
    
    
    
    #get magnitudes of model data
    # kids
    if rbvar.get() == 1:
        
        
        # fill model images with model data
        for x in range(int(size/rebins)):
            #print(x,"check")
            for y in range(int(size/rebins)):
                
                condition = (restab['id'] == str(x)+","+str(y))
                if len(restab['bayes.sdss.gp'][condition]) == 1:
                    modelg[x][y] = restab['bayes.sdss.gp'][condition][0]
                else:
                    modelg[x][y] = np.nan
                if len(restab['bayes.sdss.rp'][condition]) == 1:
                    modelr[x][y] = restab['bayes.sdss.rp'][condition][0]
                else:
                    modelr[x][y] = np.nan
                
                
                
                
        # get the smallest flux greater than zero:
        
        fming = min(modelg[modelg>0])
        #print(fming)
        fminr = min(modelr[modelr>0])
        # set all pixels with negative fluxes to fmin
        modelg[modelg<0] = fming
        modelr[modelr<0] = fminr
        
        # calculate the magnitudes
        modelg = 23.9 - 2.5*np.log10(1000*modelg/(pixscalekids*rebins**2)) 
        modelr = 23.9 - 2.5*np.log10(1000*modelr/(pixscalekids*rebins**2))
    
    # hsc
    if rbvar.get() == 2:
        # fill model images with model data
        for x in range(int(size/rebins)):
            #print(x,"check")
            for y in range(int(size/rebins)):
                
                condition = (restab['id'] == "("+str(x)+","+str(y)+")")
                if len(restab['bayes.subaru.hsc.g'][condition]) == 1:
                    modelg[x][y] = restab['bayes.subaru.hsc.g'][condition][0]
                else:
                    modelg[x][y] = np.nan
                if len(restab['bayes.subaru.hsc.r'][condition]) == 1:
                    modelr[x][y] = restab['bayes.subaru.hsc.r'][condition][0]
                else:
                    modelr[x][y] = np.nan
                
                
                
                
        # get the smallest flux greater than zero:
        
        fming = min(modelg[modelg>0])
        
        fminr = min(modelr[modelr>0])
        # set all pixels with negative fluxes to fmin
        modelg[modelg<0] = fming
        modelr[modelr<0] = fminr
        
        # calculate the magnitudes
        modelg = 23.9 - 2.5*np.log10(1000*modelg/(pixscalehsc*rebins**2)) 
        modelr = 23.9 - 2.5*np.log10(1000*modelr/(pixscalehsc*rebins**2))
    if rbvar.get() == 3:
        # fill model images with model data
        for t in range(len(restab['bayes.sdss.gp'])):
            modelg[t%int(size/rebins)][t//int(size/rebins)] = restab['bayes.sdss.gp'][t]
            modelr[t%int(size/rebins)][t//int(size/rebins)] = restab['bayes.sdss.rp'][t]
        # get the smallest flux greater than zero:
        fming = min(modelg[modelg>0])
        fminr = min(modelr[modelr>0])
        # set all pixels with negative fluxes to fmin
        modelg[modelg<0] = fming
        modelr[modelr<0] = fminr
        # calculate the magnitudes
        modelg = 23.9 - 2.5*np.log10(1000*modelg/(pixscalesdss*rebins**2)) 
        modelr = 23.9 - 2.5*np.log10(1000*modelr/(pixscalesdss*rebins**2))
    
                
    # generate dataframe for pcmd
    modelgrd = modelg - modelr
    cc = 0
    parameter = 0
    dfd2 = {"x": [], "y": [], "g-r":[], "r": [], "colorcode": [], "density":[]}
    df2 = pd.DataFrame(dfd2)
    
    # generate data frame
    logl.config(text="generating model data frame..")
    #root.update()
    
    maxd = np.sqrt(2*(len(modelr)/2)**2)
    
    
    
    isparamcc = False
    if isparamcc:
        paramcc = modelparams[0]
        maxparam = restab[paramcc].max()
        minparam = restab[paramcc].min()
    start = time.process_time()
    for x in range(0,len(modelr)):
        t = time.process_time() - start
        d = x
        if x > 5:
            sp = t/d
            logl.config(text="generating model data frame..\n"+str(x)+"/"+str(len(modelr))+" - "+str(round(100*d/len(modelr), 1))+"% - "+sectotime(sp*(len(modelr)-d)))
            root.update()
        for y in range(0,len(modelr)):
            
            if isparamcc:
                new_row = {'x':x, 'y':y, 'g-r':modelgrd[x][y], 'r':modelr[x][y], 'colorcode':cc, 'density': 0}
                df2 = df2.append(new_row, ignore_index=True)
                for param in modelparams:
                    
                    df2[param][x*len(modelr)+y] = restab[param][x*len(modelr)+y]
                    
                    cc = df2[modelparams[0]][x*len(modelr)+y]/(maxparam-minparam)
                    
            else:
                if np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) <= 0.2*maxd:
                    cc = 0
                elif np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) > 0.2*maxd and np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) <= 0.4*maxd:
                    cc = 0.25
                elif np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) > 0.4*maxd and np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) <= 0.6*maxd:
                    cc = 0.5
                elif np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) > 0.6*maxd and np.sqrt((x-len(modelr)/2)**2+(y-len(modelr)/2)**2) <= 0.8*maxd:
                    cc = 0.75
                else:
                    cc = 1
                        
                
                new_row = {'x':x, 'y':y, 'g-r':modelgrd[x][y], 'r':modelr[x][y], 'colorcode':cc, 'density': 0}
                df2 = df2.append(new_row, ignore_index=True)
    
                
    # show model pcmd
    
    fig = Figure(figsize=(6,5), dpi = 100)
    fig_canvas = FigureCanvasTkAgg(fig)
    axes = fig.add_subplot()
    axes.scatter(df2['g-r'],df2['r'],c=df2['colorcode'], marker='s', s=dsize, vmin=0, vmax=1, cmap='viridis_r')
    axes.set_ylabel('$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_xlabel('$µ_g$-$µ_r$ [mag arcsec$^{-2}$]')
    axes.set_ylim(mmax,mmin)
    axes.set_xlim(cmin,cmax)
    
    axes.set_title(title+" (model)")
    fig_canvas.get_tk_widget().grid(row=4,rowspan=5,column=0,columnspan=12)
    
    dotl = Label(root, text="", font=("Arial",1), bg='red',width=2,height=1)
    
    # show model image
    logl.config(text="plotting model image..")
    #root.update()
    
    fig2 = Figure(figsize=(6,5), dpi = 100)
    fig2_canvas = FigureCanvasTkAgg(fig2)
    fig2.canvas.mpl_connect("motion_notify_event",hover)
    fig2.canvas.mpl_connect("button_press_event",click)
    fig2.canvas.mpl_connect("button_release_event",release)
    c1 = plt.Circle((len(modelr)/2,len(modelr)/2), maxd*0.2,linewidth=1, edgecolor='yellow',facecolor="none")
    c2 = plt.Circle((len(modelr)/2,len(modelr)/2), maxd*0.4,linewidth=1, edgecolor='green',facecolor="none")
    c3 = plt.Circle((len(modelr)/2,len(modelr)/2), maxd*0.6,linewidth=1, edgecolor='mediumturquoise',facecolor="none")
    c4 = plt.Circle((len(modelr)/2,len(modelr)/2), maxd*0.8,linewidth=1, edgecolor='blue',facecolor="none")
    
    axes2 = fig2.add_subplot()
    axes2.add_patch(c1)
    axes2.add_patch(c2)
    axes2.add_patch(c3)
    axes2.add_patch(c4)
    axes2.set_ylim(0,len(modelr))
    axes2.set_xlim(0,len(modelr))
    axes2.set_title(title+" (model)")
    
    my_cmap = mcol.LinearSegmentedColormap.from_list("MyCmapName",["b","r"])
    
    my_cmap.set_bad('black')
    norm1 = mcol.Normalize(vmin=mmin,vmax=mmax)
    norm2 = mpl.colors.LogNorm(vmin=mmin, vmax=mmax)
    
    if logscalechk.get():
        fig2.colorbar(mpl.cm.ScalarMappable(norm=norm2, cmap=my_cmap), orientation='vertical')
        axes2.imshow(np.log10(modelr),cmap=my_cmap,extent=(0,len(modelr),len(modelr),0))
    else:
        fig2.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap=my_cmap), orientation='vertical')
        axes2.imshow(modelr,cmap=my_cmap,extent=(0,len(modelr),len(modelr),0))
    
    
    fig2_canvas.get_tk_widget().grid(row=4,rowspan=5,column=12,columnspan=12)
        
    #print(repr(df2))
    paramvar.set('')
    parammenu['menu'].delete(0,'end')
    for opt in restab.columns: 
        if 1 < len(Counter(restab[opt.name])):
            parammenu['menu'].add_command(label=opt.name, command=lambda x=opt.name: show(x))
    logl.config(text="model plotted.")
    #root.update()
    restabfile.close()
    flag = False
        
def mask(inp, yc, xc, r):
    for x in range(xc-r,xc+r):
        for y in range(yc-r,yc+r):
            if np.sqrt((xc-x)**2+(yc-y)**2) <= r:
                try:
                    inp[x][y] = np.nan
                except:
                    pass
    return inp

def on_closing():
    with open('config.ini', 'w') as conf:
        config_object.write(conf)
    
    root.destroy()
    
def load_config():
    global b,confdir,rebins,center,size,title,npeak,z,masks,binsize,mmax,mmin,cmax,cmin,inpdataf,thinning,dsize,convs,config_object,conff
    conff = filedialog.askopenfilename(initialdir="./", title="load config-file",filetypes=(("Configuration Files","*.ini*"),("all files","*.*")))
    config_object = ConfigParser()
    config_object.read(conff)

    params = config_object["params"]
    
    b = 0

    center = (int(params["x"]),int(params["y"]))
    size = int(params["s"])
    title = params["title"]
    npeak = params["npeak"]
    z = float(params["z"])
    masks = float(params["masks"])
    binsize = float(params["binsize"])
    mmax = float(params["mmax"])
    mmin = float(params["mmin"])
    cmax = float(params["cmax"])
    cmin = float(params["cmin"])
    dsize = float(params["dsize"])
    convs = int(params["convs"])
    rebins = int(params["rebins"])
    try:
        thinning = float(params["thinning"])
    except:
        thinning = "none"
        print("fail")
    #print(thinning)
    inpdataf = params["inpdata"]
    
    

    
    
    
    logl.config(text=os.path.basename(conff)+" loaded.")
    confdir = os.path.dirname(conff)
    #print(os.path.dirname(conff))
    
    

def hover(event):
    global flag, msklist
    if ismodel:
        if isinstance(event.xdata, float) and isinstance(event.ydata, float):
            try:
                coordl.config(text="x,y: "+str(int(event.xdata))+", "+str(int(event.ydata))+"\n"+str(round(modelr[int(event.ydata)][int(event.xdata)],5)) + " " + str(round(modelgrd[int(event.ydata)][int(event.xdata)],5)))
                modelparaml.config(text=paramvar.get()+":\n"+str(paramdata[int(event.ydata)][int(event.xdata)]))
            except:
                coordl.config(text="x,y: "+str(int(event.xdata))+", "+str(int(event.ydata))+"\n"+str(round(modelr[int(event.ydata)][int(event.xdata)],5)) + " " + str(round(modelgrd[int(event.ydata)][int(event.xdata)],5)))
            
            mx = int(event.xdata)
            my = int(event.ydata)
            if isinstance(mx,int) and isinstance(my,int) and flag == True and [mx,my] not in msklist:
                msklist.append([mx,my])
                #print(mx,my)
            
            dotl.place_forget()
    
            if mmax >= modelr[int(event.ydata)][int(event.xdata)] >= mmin and cmax >= modelgrd[int(event.ydata)][int(event.xdata)] >= cmin: 
                x = frm.winfo_x() + frm["bd"] + axes.get_position().x0*fig.get_window_extent().width + (modelgrd[int(event.ydata)][int(event.xdata)]-cmin)*axes.get_window_extent().width/(cmax-cmin) - dotl.winfo_width()/2
                y = frm.winfo_y() + frm["bd"] + axes.get_position().y0*fig.get_window_extent().height + (modelr[int(event.ydata)][int(event.xdata)]-mmin)*axes.get_window_extent().height/(mmax-mmin) - dotl.winfo_height()/2
                # start of frame + frm border + distance from border of plot to axes                  + position on the plot                                                                     + center the dot
        
                dotl.place(x=x,y=y)
            
            
                root.update()
    else:
        if isinstance(event.xdata, float) and isinstance(event.ydata, float):
            coordl.config(text=str(int(event.xdata))+" "+str(int(event.ydata))+"\n"+ str(round(inpdata[2][int(event.ydata)][int(event.xdata)],5)) + " " + str(round(grd[int(event.ydata)][int(event.xdata)],5)))
            
            mx = int(event.xdata)
            my = int(event.ydata)
            if isinstance(mx,int) and isinstance(my,int) and flag == True and [mx,my] not in msklist:
                msklist[crbvar.get()].append([mx,my])
                #print(mx,my)
                
            dotl.place_forget()
    
            if mmax >= inpdata[2][int(event.ydata)][int(event.xdata)] >= mmin and cmax >= grd[int(event.ydata)][int(event.xdata)] >= cmin: 
                x = frm.winfo_x() + frm["bd"] + axes.get_position().x0*fig.get_window_extent().width + (grd[int(event.ydata)][int(event.xdata)]-cmin)*axes.get_window_extent().width/(cmax-cmin) - dotl.winfo_width()/2
                y = frm.winfo_y() + frm["bd"] + axes.get_position().y0*fig.get_window_extent().height + (inpdata[2][int(event.ydata)][int(event.xdata)]-mmin)*axes.get_window_extent().height/(mmax-mmin) - dotl.winfo_height()/2
                # start of frame + frm border + distance from border of plot to axes                  + position on the plot                                                                     + center the dot
        
                dotl.place(x=x,y=y)
            
            
                root.update()
            
def click(event):
    global df,mx,my,shade,flag
    
    mx = int(event.xdata)
    my = int(event.ydata)
    flag = True
    
    
        
    #print(shade)
    
def release(event):
    global df,shade,flag,msklist
    
    mxr = int(event.xdata)
    myr = int(event.ydata)
    flag = False
    if mx != mxr or my != myr:
        for x in range(min(mx,mxr),max(mx,mxr)):
            for y in range(min(my,myr),max(my,myr)):
                if (x + 0.5 in shade[0]) and shade[1][shade[0].index(x + 0.5)] == y + 0.5:
                    print("double click")
        
                else:
                    df.loc[(df['x'] == y) & (df['y'] == x), ['colorcode']] = 0.5
                    shade[0].append(x + 0.5)
                    shade[1].append(y + 0.5)
                    shade[2].append(crbvar.get())
    
        
def trisum(n):
    return (n**2+n)/2

def sectotime(s):
    if s <= 60:
        cd = str(int(round(s,0))) + " s" 
    if s > 60 and s <= 3600:
        cd = str(int(s//60)) + " min " + str(int(round(s%60,0))) + " s"
    if s > 3600:
        cd = str(int(s//3600)) + " h " + str(int(round((s%3600)//60,0))) + " min"
    return cd

def showme(data, name):
    figt = Figure(figsize=(5,5), dpi = 100)
    
    axest = figt.add_subplot()
    axest.set_ylim(0,len(data))
    axest.set_xlim(0,len(data[0]))
    axest.set_title(name)
    
    axest.imshow(data,cmap='gray',vmax=max(map(max, data)),vmin=min(map(min, data)),extent=(0,len(data),len(data[0]),0))
    figt.savefig(confdir+"/"+name+".pdf")

def showmep(rdata, gdata, name):
    d = [[],[]]
    for x in range(len(rdata)):
        for y in range(len(rdata[0])):
            try:
                d[0].append(gdata[x][y] - rdata[x][y])
                d[1].append(rdata[x][y])
            except:
                print("ERROR for coordinate ",x,y," in showmep")
    #print(d)
    figt = Figure(figsize=(6,5), dpi = 100)
    
    axest = figt.add_subplot()
    axest.set_title(name)
    
    axest.scatter(d[0],d[1])
    figt.savefig(confdir+"/"+name+".pdf")
    


def showmodel(data, param, results):
    # get data from the results file
    restab = fits.open(results)[1].data
    #print(restab[param])
    
    #
    resdata = np.zeros((int(size/tilesize),int(size/tilesize)))
    #print(resdata)
    for t in range(len(restab[param])):
        resdata[t%int(size/tilesize)][t//int(size/tilesize)] = restab[param][t]
        
        
            
    
    # get the desired data cutout to show
    data0 = np.array(data[0][center[1]-int(size/2 + 0.5):center[1]+int(size/2 - 0.5), center[0]-int(size/2 + 0.5):center[0]+int(size/2 - 0.5)])
    data2 = np.array(data[2])
    
    # get magnitudes for nicer looks
    fmin = min(data0[data0>0])
    data0[data0<0] = fmin
    data0 = 2.5*np.log10(fluxmag0hsc/data0)
    
    if param == 'bayes.subaru.hsc.g':
        fmin = min(resdata[resdata>0])
        resdata[resdata<0] = fmin
        resdata = 23.9 - 2.5*np.log10(1000*resdata)
        
    
    
    # plot
    fig, axes = plt.subplots(1,2,figsize=(28,10))
    fig.suptitle(param)
    
    im1 = axes[0].imshow(data0,cmap='gray',norm=colors.LogNorm(vmin=data0.min(), vmax=data0.max()),origin='lower')
    im2 = axes[1].imshow(resdata,cmap='gray',norm=colors.LogNorm(vmin=resdata.min(), vmax=resdata.max()),origin='lower')
    cbar = fig.colorbar(im2,ax=axes[1], shrink=0.8)
    

def colorstd(rdata,gdata):
        c = []
        for x in range(len(rdata)):
            for y in range(len(rdata[0])):
                c.append(gdata[x][y]-rdata[x][y])
        return np.std(c)
    
def oversample(data,f=2):
    newd = np.array([[0.1 for y in range(len(data[0])*f)] for x in range(len(data)*f)])
    for x in range(len(data)):
        logl.config(text="oversampling with factor "+str(f)+".\n("+str(x)+"/"+str(len(data))+")")
        root.update()
        for y in range(len(data[0])):
            newd[f*x:f*x+f,f*y:f*y+f] = data[x][y]

    return newd
    
    
def rebin(data,f):
    rebinnedr = [[0 for y in range(int(len(data[0])/f))] for x in range(int(len(data)/f))]
        
    for x in range(len(data)):
        for y in range(len(data[0])):
            rebinnedr[x//f][y//f] += data[x][y] 
    for x in range(len(rebinnedr)):
        for y in range(len(rebinnedr[0])):
            rebinnedr[x][y] = rebinnedr[x][y]/f**2
    return rebinnedr
    
def myround(x, base, prec=0):
    return base*round(float(x)/float(base))


def intflux(data,x0,y0,tsize):
    datacopy = copy.deepcopy(data)
    #print("before:",datacopy[0])
    # cutout desired size
    for i in range(len(datacopy)):
        #print("filterindex:",i,"dimension:",datacopy[i].ndim)
        if datacopy[i].ndim == 2:
            # cutout
            datacopy[i] = datacopy[i][y0:y0+tsize, x0:x0+tsize]
            
            # sum fluxes
            datacopy[i] = np.sum(datacopy[i])
            #print(i,"sum of tile",datacopy[i])
    return datacopy


def create_cigale_values():
    global msklist
    if os.path.exists(confdir+'/values.dat'):
        with fits.open(confdir+"/out/results.fits") as restabfile3:
            restab3 = restabfile3[1].data
        #for i in range(len(restab3)):
         #   if restab3['best.reduced_chi_square'][i] < 0.5 or restab3['best.reduced_chi_square'][i] > 10:
          #      x = int(restab3['id'][i].split(",")[0][1:])
           #     y = int(restab3['id'][i].split(",")[1][:-1])
            #    msklist.append([x,y])
        #print(msklist)
        
        # remove the out directory if it exists
        try:
            os.remove(confdir+'/out/results.txt')
        except:
            print("could not delete results.txt")
        try:
            os.remove(confdir+'/out/results.fits')
        except:
            print("could not delete results.fits")
        try:
            os.remove(confdir+'/out/observations.txt')
        except:
            print("could not delete observations.txt")
        try:
            os.remove(confdir+'/out/observations.fits')
        except:
            print("could not delete observations.fits")
        try:
            os.remove(confdir+'/out/pcigale.ini')
        except:
            print("could not delete pcigale.ini")
        try:
            os.remove(confdir+'/out/pcigale.ini.spec')
        except:
            print("could not delete pcigale.ini.spec")
        try:
            os.rmdir(confdir+'/out')
        except:
            print("could not delete /out")
            
        
                    
            
        
        
        # delete specified pixels from values.dat
        
        with open("values.dat", "r") as valuesfile:
            pixel_data = valuesfile.readlines()

            
        updated_pixel_data = []
        line = 0
        delflag = False
        # go through alle the lines
        while line < len(pixel_data):
            # for each line, check if one of the pixels in msklist is in the line
            for p in range(len(msklist)):
                # if so,
                if "("+str(msklist[p][1])+","+str(msklist[p][0])+")\t" in str(pixel_data[line]):
                    print(str(msklist[p][1])+","+str(msklist[p][0])+" found")
                    # delete that line
                    print(pixel_data.pop(line))
                    # flag that a line has been deletetd
                    delflag = True
                    # leave the loop, so that the rest of msklist doe not get compared unnecessarily
                    break
            # check if a line has been deleted and only go to the next line if not
            if delflag == True:
                delflag = False
            else:
                line += 1

                

        # Write the updated configuration back to the file
        with open("values.dat", "w") as valuesfile:
            valuesfile.writelines(pixel_data)
            
        logl.config(text="deleted mask buffer pixels from existing values.dat file.")
        root.update()
        
    else:
        logl.config(text="creating data table for cigale..")
        root.update()
        # initialize data table and tiledata
        rawdata = []
        data = []
    
        # which filters should be used?
        if rbvar.get() == 1:
            filters = kids_filters
        if rbvar.get() == 2:
            filters = hsc_filters
        if rbvar.get() == 3:
            filters = sdss_filters
        
        for filt in filters:
            # open .fits file for filter
            rawdata.append(fits.open(inpdataf+""+filt+".fits"))
            data.append(0)
            # get header for filter
            head = fits.open(inpdataf+filt+".fits")[0].header
        
            # get background variance from datapoints fainter than mmax
        
            if rbvar.get() == 2:
                background = np.array(rawdata[-1][1].data)
                background = background[background < fluxmag0hsc/10**(mmax/2.5)]
            if rbvar.get() == 1:
                background = np.array(rawdata[-1][0].data)
                background = background[background < 10**(mmax/-2.5)]
            std = np.std(background)
            var = (10**(1.4/2.5 + np.log10(std)/2.5)/1000)**2
            #print("bg error:",var**12)
        
        
            if rbvar.get() == 3:
                tab = Table(names=sdss_filternames) 
                # get data from hdul
                rawdata[-1] = rawdata[-1][0].data
                # get the smallest flux greater than zero:
                fmin = min(rawdata[-1][rawdata[-1]>0])
                # set all pixels with negative fluxes to fmin
                rawdata[-1][rawdata[-1]<0] = fmin
                # get fluxes in mJy
                rawdata[-1] = 10**(1.4/2.5 + np.log10(rawdata[-1])/2.5)/1000
                # get error in mJy
                rawdata.append(10**(1.4/2.5 + np.log10(np.sqrt(sdss_errors[filters.index(filt)]))/2.5)/1000)
            # hsc
            if rbvar.get() == 2: 
                rawdata[0].info()
                
                names = ""
                for filt in hsc_filternames:
                    names = names + "\t" + filt
                names = names +"\n"
                names = names.replace('\t','',1)
                lines = [names]
            
                # get data from hdul
                data[-1] = rawdata[-1][1].data
                data.append(rawdata[-1][3].data)
            
                # get fluxes in mJy
                data[-2] = 10**(23.9/2.5)*data[-2]/fluxmag0hsc/1000
                # get error in mJy
                data[-1] = 10**(23.9/2.5)*data[-1]/fluxmag0hsc/1000
                #rawdata.append(10*var)
            
            # kids
            if rbvar.get() == 1:
                names = ""
                for filt in kids_filternames:
                    names = names + "\t" + filt
                names = names +"\n"
                names = names.replace('\t','',1)
                lines = [names]
            
                # get data from hdul
                rawdata[-1] = rawdata[-1][0].data
            
                # get fluxes in mJy
                rawdata[-1] = 0.001*rawdata[-1]*10**(23.9/2.5)
                # get error in mJy
                #rawdata.append(10**(23.9/2.5)*np.sqrt(head['BGVAR'])/fluxmag0hsc/1000)
                rawdata.append(10*var)
            
        if alnchk.get():
            logl.config(text="aligning images..")
            root.update()
            # align other filters to first filter
            #rawdata[0] = rawdata[0].byteswap().newbyteorder()
            for i in range(2,len(data),2):
                
                #rawdata[i] = rawdata[i].byteswap().newbyteorder()
                data[i] = np.array(aa.register(data[i],data[0],detection_sigma=2)[0])
            data[0] = np.array(data[0])
       
        print("before mask:",np.count_nonzero(np.isnan(data[0])))

        
        
        #AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAHHHHHHHHHHHHHHHHHH 
        
        
        
        # split the string from the config file to a list
        peakl = [x for x in npeak.split()]
    
        # split each element of the list into 1. x coordinate, 2. y coordinate, 3. mask size
        for i in range(0,len(peakl)):
            peakl[i] = [int(x) for x in peakl[i].split(",")]
        
        if len(peakl) > 0 and rebinchk.get():
            for i in range(0,len(peakl)):
                for j in range(0,len(data),2):
                
                    print("mask:",i,"filter:",j)
                    data[j] = mask(data[j],int(peakl[i][0])+center[0]-int(size/2), int(peakl[i][1])+center[1]-int(size/2),int(peakl[i][2]))
                    
        else:
            for i in range(0,len(peakl)):
                for j in range(0,len(cutdata),2):
                    cutdata[j] = mask(cutdata[j],center[0]-int(size/2)+int(peakl[i][0]), center[1]-int(size/2)+int(peakl[i][1]),int(peakl[i][2]))
        tiledata = []
        print("after mask:",np.count_nonzero(np.isnan(data[0])))
        corner = [center[0]-int(size/2),center[1]-int(size/2)]
        start = time.process_time()
        for x in range(int(size/rebins)):
            for y in range(int(size/rebins)):
                t = time.process_time() - start
                d = x
                if x > 5:
                    sp = t/d
                    logl.config(text="creating data table for cigale.. "+str(int(x*(size/rebins)+y))+"/"+str(int((size/rebins)**2))+" - "+str(round(100*d/int(size/rebins), 1))+"% - "+sectotime(sp*(int(size/rebins)-d)))
                root.update()
            
                tile = intflux(data.copy(), corner[0] + x * rebins, corner[1] + y * rebins, rebins)
            
            
                if tile[2] >= (pixscalehsc*rebins**2)*10**((23.9 - mmax)/2.5)/1000:
                    #print(x,y,23.9 - 2.5*np.log10(1000*tile[0]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[1]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[2]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[3]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[4]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[5]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[6]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[7]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[8]/(pixscalehsc*rebins**2)))
                    #print(23.9 - 2.5*np.log10(1000*tile[9]/(pixscalehsc*rebins**2)))
            
                    tiledata.append(tile)
            
            
                    line = "("+str(y)+","+str(x) + ")\t" + str(z) + "\t"
                    for i in range(len(data)):
                        if i%2 == 0:
                            line = line + str(tiledata[-1][i]) + "\t"
                        else:
                            line = line + str(np.sqrt(tiledata[-1][i])/rebins**2) + "\t"
                    line = line +"\n"
                    #print(line)
                    lines.append(line)
            
        
            
        with open(confdir+'/values.dat','w') as f:
            f.writelines(lines)
    
    
        logl.config(text="created values.dat")
        root.update()
    
def find_redshift():
    if os.path.exists(confdir+'/redshifts.txt') == 0:
        #create table
        tab = Table(names=["z","reduced_chi^2_sum"]) 
        ascii.write(tab, confdir+'/redshifts.txt', overwrite=True, delimiter='\t')
    for i in range(2554,6154,200):
        z = 0.00001*i
        logl.config(text="calculating reduced chi^2 for z = "+str(z))
        root.update()
        # remove the out directory if it exists
        try:
            os.remove(confdir+'/out/results.txt')
        except:
            print("could not delete results.txt")
        try:
            os.remove(confdir+'/out/results.fits')
        except:
            print("could not delete results.fits")
        try:
            os.remove(confdir+'/out/observations.txt')
        except:
            print("could not delete observations.txt")
        try:
            os.remove(confdir+'/out/observations.fits')
        except:
            print("could not delete observations.fits")
        try:
            os.remove(confdir+'/out/pcigale.ini')
        except:
            print("could not delete pcigale.ini")
        try:
            os.remove(confdir+'/out/pcigale.ini.spec')
        except:
            print("could not delete pcigale.ini.spec")
        try:
            os.rmdir(confdir+'/out')
        except:
            print("could not delete /out")
        
            #break
        # change the z value in the pcigale.ini file
        
        with open("pcigale.ini", "r") as configfile:
            config_data = configfile.readlines()

            
        updated_config_data = []
        for line in config_data:
            if "redshift =" in line:
                line = "    redshift = "+str(z)+"\n"
                #print("line found")

            updated_config_data.append(line)

        # Write the updated configuration back to the file
        with open("pcigale.ini", "w") as configfile:
            configfile.writelines(updated_config_data)
            
        cig_conf = Configuration()
        # run cigale
        pcigale.run(cig_conf)
        # open the results file
        
        restabfile = fits.open(confdir+"/out/results.fits",memmap=False)
        restab = restabfile[1].data
        row = [restab['best.universe.redshift'][0],sum(restab['best.reduced_chi_square'])]
        print("ch^2 sum:",sum(restab['best.reduced_chi_square']))
        #tab.add_row(row)
        #check_results()
        restabfile.close()
        table_frame = pd.read_csv(confdir+'/redshifts.txt', delimiter='\t')
        table_frame.loc[len(table_frame)] = row
        table_frame.to_csv(confdir+'/redshifts.txt', sep='\t', index=False)
        
    
    
def check_results():  
    global foldername
    lines = []
    restab2 = fits.open(confdir+"/out/results.fits")[1].data
    
    print("chosen param:",paramvar.get())
    # age histogram
    #agemean = np.mean(restab2[paramvar.get()][restab2[paramvar.get()]<restab2[paramvar.get()].max()])
    fig9 = Figure(figsize=(6,5), dpi = 100)
    axes9 = fig9.add_subplot()
    
    axes9.hist(restab2['best.sfh.age'],13779,log=True)
    #axes9.axvline(x=agemean,color='red')
    axes9.set_ylim(0,200)
    axes9.set_xlim(0,14000)
    axes9.set_title(title+" Age")
    try:    
        fig9.savefig(confdir+"/"+foldername+"/"+title+"_age_hist.pdf")
    except:
        # Get the current date and time
        current_time = datetime.datetime.now()

        # Format the current time as a string
        foldername = current_time.strftime("%Y-%m-%d_%H-%M-%S")

        # Create a folder with the formatted name
        os.mkdir(foldername)
        
        fig9.savefig(confdir+"/"+foldername+"/"+title+"_age_hist.pdf")
    if len(paramvar.get()) > 1:
        # parameter histogram
        fig8 = Figure(figsize=(6,5), dpi = 100)
        axes8 = fig8.add_subplot()
        axes8.hist(restab2[paramvar.get()],bins=np.arange(0,14000, 100))
        axes8.set_title(title+"_"+str(paramvar.get()))
    
        fig8.savefig(confdir+"/"+foldername+"/"+title+"_"+str(paramvar.get())+"_hist.pdf")
    
    
    
        
    # error histogram
    fig10 = Figure(figsize=(6,5), dpi = 100)
    axes10 = fig10.add_subplot()
    
    axes10.hist(restab2['best.reduced_chi_square'],m.ceil(np.nanmax(restab2['best.reduced_chi_square'])))
    axes10.axvline(x=1,color='red')
    axes10.set_title(title+" Reduced Chi^2")
        
    fig10.savefig(confdir+"/"+foldername+"/"+title+"_error_hist.pdf")
        
        
        
        
    # radial age profile (arsec)
    radialagedata = radial_profile(restab2,'best.sfh.age')
    fig11 = Figure(figsize=(6,5), dpi = 100)
    axes11 = fig11.add_subplot()
    axes11.plot(radialagedata[0],radialagedata[2])
    #axes11.set_xlim(0,50)
    axes11.set_ylim(0,14000)
    axes11.set_ylabel("Age [Myr]")
    axes11.set_xlabel("Radius [arcsec]")
    axes11.set_title(title+" Radial Age Profile")
    fig11.savefig(confdir+"/"+foldername+"/"+title+"_radial_age_arcsec.pdf")
    
    # radial age profile (kpc)
    fig12 = Figure(figsize=(6,5), dpi = 100)
    axes12 = fig12.add_subplot()
    axes12.plot(radialagedata[1],radialagedata[2])
    #axes12.set_xlim(0,40)
    axes12.set_ylim(0,14000)
    axes12.set_ylabel("Age [Myr]")
    axes12.set_xlabel("Radius [kpc]")
    axes12.set_title(title+" Radial Age Profile")
    fig12.savefig(confdir+"/"+foldername+"/"+title+"_radial_age_kpc.pdf")
    
    # radial mag profile (arsec)
    radialmagdata = radial_profile(restab2,'bayes.subaru.hsc.r')
    for i in range(len(radialmagdata[2])):
        radialmagdata[2][i] = 23.9 - 2.5*np.log10(1000*radialmagdata[2][i]/(pixscalehsc*rebins**2))
        
    fig13 = Figure(figsize=(6,5), dpi = 100)
    axes13 = fig13.add_subplot()
    axes13.plot(radialmagdata[0],radialmagdata[2])
    #axes13.set_xlim(0,50)
    axes13.set_ylim(mmax,mmin)
    axes13.set_ylabel("$µ_r$ [mag arcsec$^{-2}$]")
    axes13.set_xlabel("Radius [arcsec]")
    axes13.set_title(title+" Radial Surface Brightness")
    fig13.savefig(confdir+"/"+foldername+"/"+title+"_radial_mag_arcsec.pdf")
    
    # radial age profile (kpc)
    fig14 = Figure(figsize=(6,5), dpi = 100)
    axes14 = fig14.add_subplot()
    axes14.plot(radialmagdata[1],radialmagdata[2])
    #axes14.set_xlim(0,40)
    axes14.set_ylim(mmax,mmin)
    axes14.set_ylabel("$µ_r$ [mag arcsec$^{-2}$]")
    axes14.set_xlabel("Radius [kpc]")
    axes14.set_title(title+" Radial Surface Brightness")
    fig14.savefig(confdir+"/"+foldername+"/"+title+"_radial_mag_kpc.pdf")
    
    # save plots
    save_plot()
    
    # results text file
    
    # iterate over all columns (fit parameters)
    for col in restab2.columns:
        # only consider those which are discrete
        if len(Counter(restab2[col.name])) < 100 or col.name == paramvar.get():
            # name of the parameter
            print(col.name)
            row = [col.name]
            # iterate over all parameter values
            for i in range(len(Counter(restab2[col.name]).most_common())):
                # append the parameter value
                row.append(Counter(restab2[col.name]).most_common()[i][0])
                # append the number of occurences of that parameter value
                row.append(Counter(restab2[col.name]).most_common()[i][1])
            lines.append(str(row)+"\n")
                
            #print(col.name,Counter(restab[col.name]))
    #print(restab2['best.reduced_chi_square'])
    
    lines.append("radial age profile: "+str(radialagedata)+"\n")
    lines.append("radial mag profile: "+str(radialmagdata))
    
    chisum = round(np.median(restab2['best.reduced_chi_square']),2)
            
    
    with open(confdir+'/'+foldername+"/"+'result_'+str(chisum)+".txt",'w') as f:
        f.writelines(lines)
    
    
def show(x):
    paramvar.set(x)  
    
def showparam():
    global paramdata, fig2
    paramdata = np.zeros((int(size/rebins),int(size/rebins)))
    for x in range(int(size/rebins)):
        for y in range(int(size/rebins)):
            condition = (restab['id'] == "("+str(x)+","+str(y)+")")
            if len(restab[paramvar.get()][condition]) == 1:
                paramdata[x][y] = restab[paramvar.get()][condition][0]
            else:
                paramdata[x][y] = np.nan
                
    #for t in range(len(restab[paramvar.get()])):
     #       paramdata[t%int(size/rebins)][t//int(size/rebins)] = restab[paramvar.get()][t]
    fig2 = Figure(figsize=(6,5), dpi = 100)
    fig2_canvas = FigureCanvasTkAgg(fig2)
    fig2.canvas.mpl_connect("motion_notify_event",hover)
    fig2.canvas.mpl_connect("button_press_event",click)
    fig2.canvas.mpl_connect("button_release_event",release)
    
    
    axes2 = fig2.add_subplot()
    axes2.set_ylim(0,len(modelr))
    axes2.set_xlim(0,len(modelr))
    axes2.set_title(title+" "+paramvar.get())
    #print(paramdata.max())
    my_cmap = mcol.LinearSegmentedColormap.from_list("MyCmapName",["b","r"])
    
    my_cmap.set_bad('black')
    
    if paramvar.get() == 'best.sfh.age':
        norm1 = mcol.Normalize(vmin=np.nanmin(paramdata),vmax=14000)
        norm2 = mpl.colors.LogNorm(vmin=np.nanmin(paramdata), vmax=14000)
    else:
        norm1 = mcol.Normalize(vmin=np.nanmin(paramdata),vmax=np.nanmax(paramdata))
        norm2 = mpl.colors.LogNorm(vmin=np.nanmin(paramdata), vmax=np.nanmax(paramdata))
    if logscalechk.get():
        fig2.colorbar(mpl.cm.ScalarMappable(norm=norm2, cmap=my_cmap), orientation='vertical')
        axes2.imshow(np.log10(paramdata),cmap=my_cmap,extent=(0,len(modelr),len(modelr),0))
    else:
        fig2.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap=my_cmap), orientation='vertical')
        axes2.imshow(paramdata,cmap=my_cmap,extent=(0,len(modelr),len(modelr),0))
    
    
    fig2_canvas.get_tk_widget().grid(row=4,rowspan=5,column=12,columnspan=12)
    
def color_image():
    hdui = fits.open(inpdataf+"I"+".fits")[1]
    wcs = WCS(hdui.header)
    
    cutouti = Cutout2D(hdui.data, position=(center[0],center[1]), size=size, wcs=wcs)
    hdui.data = cutouti.data
    hdui.header.update(cutouti.wcs.to_header())
    hdui.writeto("itest.fits", overwrite=True)
    
    hdur = fits.open(inpdataf+"R"+".fits")[1]
    wcs = WCS(hdur.header)
    
    cutoutr = Cutout2D(hdur.data, position=(center[0],center[1]), size=size, wcs=wcs)
    hdur.data = cutoutr.data
    hdur.header.update(cutoutr.wcs.to_header())
    hdur.writeto("rtest.fits", overwrite=True)
    
    hdug = fits.open(inpdataf+"G"+".fits")[1]
    wcs = WCS(hdug.header)
    
    cutoutg = Cutout2D(hdug.data, position=(center[0],center[1]), size=size, wcs=wcs)
    hdug.data = cutoutg.data
    hdug.header.update(cutoutg.wcs.to_header())
    hdug.writeto("gtest.fits", overwrite=True)
    
    imagesRGB = {"R": ["itest.fits[1]"], \
                 "G": ["rtest.fits[1]"], \
                 "B": ["gtest.fits[1]"]}
    noiselums = {'R': 0.5, 'G': 0.5, 'B': 0.5}
      
    trilogy.Trilogy(infile = None, samplesize = 20000, stampsize = 20000, maxstampsize = 20000, \
                deletetests = 1, deletefilters = 1, testfirst = 0, showwith = "PIL", \
                mode = 'RGB', imagesorder = 'RGB', imagesRGB = imagesRGB, noiselums = noiselums, images = None, \
                outname = title+'_color_image', outdir=foldername+'/', satpercent = 0.0009, noiselum = 0.5, noisesig = 50, \
                noisesig0 = 10, correctbias = 0, colorsatfac = 1, combine = 'sum', show = False).run()
    os.remove("itest.fits")
    os.remove("rtest.fits")
    os.remove("gtest.fits")
    
def radial_profile(table,parameter):
    ccoord = [int(size/(2*rebins)-0.5),int(size/(2*rebins)-0.5)]
    agedata = []
    coordlisth = []
    for r in range(int(size/(2*rebins))):
        agedatar = []
        coordlist = []
        print("r =",r)
        for step in range(1000):
            phi = step*2*np.pi/1000
            if [int((r+0.5)*np.cos(phi)) + ccoord[0],int((r+0.5)*np.sin(phi)) + ccoord[1]] not in coordlist and [int((r+0.5)*np.cos(phi)) + ccoord[0],int((r+0.5)*np.sin(phi)) + ccoord[1]] not in coordlisth:
                coordlist.append([int((r+0.5)*np.cos(phi)) + ccoord[0],int((r+0.5)*np.sin(phi)) + ccoord[1]])
        for i in range(len(coordlist)):
            if "("+str(coordlist[i][0])+","+str(coordlist[i][1])+")\t" in table['id']:
                agedatar.append(table[parameter][table['id'] == "("+str(coordlist[i][0])+","+str(coordlist[i][1])+")\t"])
        coordlisth = coordlist
        agedata.append(np.median(agedatar))
        print(r, agedata[-1])
    agedata = [[],[],agedata]
    for i in range(len(agedata[2])):
        rarcsec = rebins*i*np.sqrt(pixscalehsc)
        agedata[0].append(rarcsec)
        agedata[1].append(((table['best.universe.luminosity_distance'][0]*rarcsec*np.pi)/(3600*180*(1+table['best.universe.redshift'][0])**2))/3.0857E+19)
    return agedata
    
    
      
        

    






#buttons
plotb = Button(root, text="plot", command=plot,width=12,bg='#e7e7e7')
plotb.grid(row=0, rowspan=1, column=16, columnspan=1, padx='6', pady='6')

saveb = Button(root, text="save plots", command=save_plot,width=12,bg='#e7e7e7')
saveb.grid(row=1, rowspan=1, column=16, columnspan=1, padx='6', pady='6')

loadcb = Button(root, text="load config file", command=load_config,width=12,bg='#e7e7e7')
loadcb.grid(row=2, rowspan=1, column=16, columnspan=1, padx='6', pady='6')

#redshiftb = Button(root, text="redshift", command=find_redshift,width=12,bg='#e7e7e7')
#redshiftb.grid(row=3, rowspan=1, column=16, columnspan=1, padx='6', pady='6')

replotb = Button(root, text="replot", command=replot,width=12,bg='#e7e7e7')
replotb.grid(row=3, rowspan=1, column=17, columnspan=1, padx='6', pady='6')

plotmodelb = Button(root, text="plot model", command=plotmodel,width=12,bg='#e7e7e7')
plotmodelb.grid(row=0, rowspan=1, column=17, columnspan=1, padx='6', pady='6')

plotmodelb = Button(root, text="create cigale data", command=create_cigale_values,width=12,bg='#e7e7e7')
plotmodelb.grid(row=1, rowspan=1, column=17, columnspan=1, padx='6', pady='6')

check_resultsb = Button(root, text="check results", command=check_results,width=12,bg='#e7e7e7')
check_resultsb.grid(row=2, rowspan=1, column=17, columnspan=1, padx='6', pady='6')

showparamb = Button(root, text="show param", command=showparam,width=12,bg='#e7e7e7')
showparamb.grid(row=0, rowspan=1, column=20, columnspan=8, padx='6', pady='4',sticky='w')

# label

modelparaml = Label(root, text="",bg='#e7e7e7',anchor='center')
modelparaml.grid(row=2, rowspan=2, column=18, columnspan=4, padx='6', pady='6', sticky='w')


logl = Label(root, text="",bg='#e7e7e7')
logl.grid(row=16, rowspan=2, column=0, columnspan=2, padx='6', pady='6', sticky='w')

coordl = Label(root, text="",bg='#e7e7e7',anchor='center')
coordl.grid(row=1, rowspan=2, column=18, columnspan=4, padx='6', pady='6', sticky='w')


frm = Frame(root, bd=3, relief=SUNKEN, width=607, height=507)
frm.grid(row=4, rowspan=5, column=0, columnspan=12, padx='6', pady='4')

frm2 = Frame(root, bd=3, relief=SUNKEN, width=607, height=507)
frm2.grid(row=4, rowspan=5, column=12, columnspan=12, padx='6', pady='4')






# checkboxes
alnchk = IntVar(value=1)
Checkbutton(root, text="align images", variable=alnchk,bg='#e7e7e7').grid(row=0, column = 8, columnspan=4)

convchk = IntVar(value=1)
Checkbutton(root, text="convolve images", variable=convchk,bg='#e7e7e7').grid(row=1, column = 8,columnspan=4)

#bbchk = IntVar()
#Checkbutton(root, text="plot backbone", variable=bbchk,bg='#e7e7e7').grid(row=2, column = 8,columnspan=4)

rebinchk = IntVar(value=1)
Checkbutton(root, text="rebin", variable=rebinchk,bg='#e7e7e7').grid(row=0, column = 12,columnspan=4)

#thinchk = IntVar()
#Checkbutton(root, text="density map", variable=thinchk,bg='#e7e7e7').grid(row=1, column = 12,columnspan=4)

logscalechk = IntVar()
Checkbutton(root, text="param logscale", variable=logscalechk,bg='#e7e7e7').grid(row=2, column = 12,columnspan=4)

#myalignchk = IntVar()
#Checkbutton(root, text="my align", variable=myalignchk,bg='#e7e7e7').grid(row=3, column = 8,columnspan=4)

# radiobuttons

rbvar = IntVar(value=2)


kidsrb = Radiobutton(root, text="KiDS", variable=rbvar, value=1,bg='#e7e7e7')
hscrb = Radiobutton(root, text="HSC", variable=rbvar, value=2,bg='#e7e7e7')
sdssrb = Radiobutton(root, text="SDSS", variable=rbvar, value=3,bg='#e7e7e7')

kidsrb.grid(row=0,column=6,columnspan=2)
hscrb.grid(row=1,column=6,columnspan=2)
sdssrb.grid(row=2,column=6,columnspan=2)


crbvar = IntVar(value=0)


yrb = Radiobutton(root, text="yellow", variable=crbvar, value=0,bg='#e7e7e7')
grb = Radiobutton(root, text="green", variable=crbvar, value=1,bg='#e7e7e7')
mrb = Radiobutton(root, text="turquoise", variable=crbvar, value=2,bg='#e7e7e7')
brb = Radiobutton(root, text="blue", variable=crbvar, value=3,bg='#e7e7e7')

yrb.grid(row=0,column=4,columnspan=2)
grb.grid(row=1,column=4,columnspan=2)
mrb.grid(row=2,column=4,columnspan=2)
brb.grid(row=3,column=4,columnspan=2)


paramvar = StringVar(root)
paramvar.set('nil') # default value

parammenu = OptionMenu(root, paramvar, 'nil')
parammenu.grid(row=0, rowspan=1, column=18, columnspan=2, padx='6', pady='4',sticky='w')





pixscalekids = 0.04 #arcsec^2/pixel
pixscalehsc = 0.028224
pixscalesdss = 0.156816
fluxmag0hsc = 63095734448.01944
backbones = []


root.mainloop()




